# Import the required packages


In [ ]:
import os
from typing import Any
from torch.nn import (
    BatchNorm2d, Conv2d, Flatten, Linear, LogSoftmax, MaxPool2d, Module,
    ReLU, Tanh, NLLLoss
)

from torchvision.datasets import FashionMNIST

from torch import device as torch_device
from torch.nn import Flatten, LogSoftmax, Sigmoid
from aihwkit.nn import AnalogConv2dMapped, AnalogLinearMapped, AnalogSequential
from aihwkit.simulator.presets.utils import PresetIOParameters
from aihwkit.simulator.configs import InferenceRPUConfig
from aihwkit.simulator.configs.utils import  MappingParameter, WeightClipType, WeightModifierType, WeightRemapType

from aihwkit.experiments import BasicTraining
from aihwkit.experiments.runners import CloudRunner


## RPU Configuration
Now that the package is installed and running, we can start working on creating the LeNet5 network.

AIHWKIT offers different Analog layers that can be used to build a network, including AnalogLinear and AnalogConv2d which will be the main layers used to build the present network. 
In addition to the standard input that are expected by the PyTorch layers (in_channels, out_channels, etc.) the analog layers also expect a rpu_config input which defines various settings of the RPU tile. 

Through the rpu_config parameter the user can specify many of the hardware specs such as: device used in the cross-point array, bit used by the ADC/DAC converters, noise values and many other. 

Additional details on the RPU configuration can be found at https://aihwkit.readthedocs.io/en/latest/using_simulator.html#rpu-configurations

For this particular case we will define a RPU which uses RRAM devices in the cross-point array:

In [ ]:
def create_rpu_config():

    from aihwkit.simulator.presets import ReRamSBPreset

    rpu_config=ReRamSBPreset()

    return rpu_config

We can now use this rpu_config as input of the network model:

In [ ]:
from torch.nn import Tanh, MaxPool2d, LogSoftmax, Flatten
from aihwkit.nn import AnalogConv2d, AnalogLinear, AnalogSequential

def create_analog_lenet5_network():
    """Returns a LeNet5 inspired analog model."""

    N_CLASSES = 10
    rpu_config = create_rpu_config()
    channel = [16, 32, 512, 128]
    model = AnalogSequential(
        AnalogConv2d(in_channels=1, out_channels=channel[0], kernel_size=5, stride=1,
                     rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        AnalogConv2d(in_channels=channel[0], out_channels=channel[1], kernel_size=5, stride=1,
                     rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        Tanh(),
        Flatten(),
        AnalogLinear(in_features=channel[2], out_features=channel[3], rpu_config=rpu_config),
        Tanh(),
        AnalogLinear(in_features=channel[3], out_features=N_CLASSES, rpu_config=rpu_config),
        LogSoftmax(dim=1)
    )

    return model


## Create an instance of the BasicTraining class for a  training experiment.

The created Experiment contains the definition of the operation to be performed, but is not executed automatically. That is the role of the Runners.


In [ ]:
DATASET = FashionMNIST
MODEL = create_analog_lenet5_network()
BATCH_SIZE = 8
LOSS_FUNCTION = NLLLoss
EPOCHS = 2
LEARNING_RATE = 0.05
MAX_ELEMENTS = 2000

my_experiment = BasicTraining(
    dataset=DATASET,
    model = MODEL,
    batch_size = BATCH_SIZE,
    loss_function = LOSS_FUNCTION,
    epochs = EPOCHS,
    learning_rate = LEARNING_RATE
)

## Create a cloud runner, run the training experiment

A Runner is the object that controls the execution of an Experiment, setting up the environment and providing a convenient way of starting it and retrieving its results. In the code snippet below we will create a cloud runner instance.


In [ ]:
# Create a cloud runner using cuda device
cloud_runner = CloudRunner()
NAME = os.getenv('JNAME')
if not NAME:
    NAME = 'CLI_training'
cloud_experiment = cloud_runner.run(my_experiment, name=NAME, device='gpu')


## Display the experiment ID

In [ ]:
import json
print('cloud_experiment: ', cloud_experiment)